<a href="https://colab.research.google.com/github/JerryChenz/InvestmentManagement/blob/master/stock_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Set inputs

We load the sample dataset from the my github repository, and display the dataset in pandas.

In [25]:
import pandas as pd
summary_url = 'https://raw.githubusercontent.com/JerryChenz/InvestmentManagementOpen/main/financial_models/Opportunities/Screener/screener_summary_latest.csv'
df = pd.read_csv(summary_url)
# Preparing the data
df['Last_fy'] = pd.to_datetime(df['Last_fy'])
# Missing Forex_rate
mop_hkd = (df["Price_currency"] == "HKD") & (df["Reporting_Currency"] == "MOP")
twd_hkd = (df["Price_currency"] == "HKD") & (df["Reporting_Currency"] == "TWD")
twd_usd = (df["Price_currency"] == "USD") & (df["Reporting_Currency"] == "TWD")
df.loc[mop_hkd, "Fx_rate"] = 0.98
df.loc[mop_hkd, "Fx_rate"] = 0.26
df.loc[twd_usd, "Fx_rate"] = 7.81
# exclude minor countries
df = df[df['Fx_rate'].notna()]
df = df.fillna(0)

# capitalization in reporting currency
capitalization_price = df['Price'] * df['Shares']
capitalization_report = capitalization_price / df['Fx_rate']
# debt
total_debt = df['CurrentDebtAndCapitalLeaseObligation'] + df['LongTermDebtAndCapitalLeaseObligation']
# more easily realizable non-operating assets
monetary_assets = df['CashAndCashEquivalents'] + df['OtherShortTermInvestments'] + df['InvestmentinFinancialAssets']
# liquidity_coverage_ratio
core_lcr = df['CashAndCashEquivalents'] / df['CurrentLiabilities']
lcr = monetary_assets / df['CurrentLiabilities']
# leverage ratio
current_ratio = df['CurrentAssets'] / df['CurrentLiabilities']
debt_ratio = total_debt / (df['TotalEquityGrossMinorityInterest'] - df['MinorityInterest'])
# less easily realizable non-operating assets
fixed_nonop_assets = (df['InvestmentProperties'] + df['LongTermEquityInvestment']) * 0.5
total_nonop_assets = monetary_assets + fixed_nonop_assets
enterprise_value = capitalization_report + total_debt + df['MinorityInterest'] - df['CashAndCashEquivalents'] - total_nonop_assets
# dividend rate & buyback rate
dividend_rate = df['Dividend'] / df['Price']
buyback_rate = df['Buyback'] / df['Price']
# Net PPE/ Sales
ppe_multiple = df['NetPPE'] / df['TotalRevenue']

display(df)

,Ticker,Name,Exchange,Price,Price_currency,Shares,Reporting_Currency,Fx_rate,Dividend,Buyback,...,EBIT,EbitMargin,Avg_ebit_margin,Avg_ebit_growth,InterestExpense,NetIncomeCommonStockholders,NetMargin,Avg_net_margin,Avg_NetIncome_growth,Years_of_data
0,000036.SZ,CHINA UNION HOLD L,SHZ,4.0100,CNY,1483929984,CNY,1.000000,0.000000,0.000000,...,1.034388e+09,54.96,61.500000,-30.57,5.205453e+07,4.063153e+08,21.59,22.590000,-29.43,3.0
1,000045.SZ,SHN TEXTILE HLDGS,SHZ,10.6700,CNY,457022016,CNY,1.000000,0.000000,0.000000,...,3.237886e+08,14.12,10.793333,53.97,1.430628e+07,6.116238e+07,2.67,1.783333,76.74,3.0
2,000088.SZ,SHN YAN TIAN PORT,SHZ,4.9900,CNY,2249159936,CNY,1.000000,0.000000,0.000000,...,2.232715e+08,32.84,31.630000,15.78,7.878430e+07,4.614137e+08,67.87,67.400000,13.39,3.0
3,000096.SZ,SHENZHEN GUANGJU E,SHZ,8.5000,CNY,528000000,CNY,1.000000,0.000000,0.000000,...,1.078495e+08,6.66,6.390000,7.97,1.005905e+05,6.770438e+07,4.18,7.483333,-21.90,3.0
4,000099.SZ,CITIC OFFSHORE HEL,SHZ,7.2800,CNY,775769984,CNY,1.000000,0.000000,0.000000,...,4.122275e+08,24.53,25.100000,5.79,6.176388e+07,2.464817e+08,14.67,14.056667,8.07,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2803,ZD,"Ziff Davis, Inc.",NMS,82.8000,USD,47191600,USD,1.000000,0.000000,1.659766,...,2.788430e+08,19.68,23.540000,-4.88,7.202300e+07,4.963880e+08,35.04,20.266667,100.26,3.0
2804,ZG,"Zillow Group, Inc.",NMS,39.6300,USD,58197100,USD,1.000000,0.000000,5.194365,...,2.281910e+08,2.80,9.256667,40.67,1.919100e+08,-5.277770e+08,-6.48,-7.486667,89.32,3.0
2805,ZIM,ZIM Integrated Shipping Service,NYQ,17.5849,USD,120047000,USD,1.000000,4.468150,0.000000,...,5.836800e+09,54.40,25.720000,554.54,1.688980e+08,4.640305e+09,43.25,18.560000,-1079.03,3.0
2806,ZTO,ZTO Express (Cayman) Inc.,NYQ,28.2000,USD,809732992,CNY,0.147437,1.672118,4.705978,...,4.713508e+09,15.50,18.333333,-2.41,1.265030e+08,4.754827e+09,15.64,19.466667,-6.87,3.0


# Step 2: Screening Criteria

We can screen using different sets of conditions, then merge them later.

In [26]:
# common fitlering conditions: 
# 1. Reasonable leverage and Good Liqudity
common_1 = (core_lcr >= 0.6) & (lcr >= 0.8)
common_2 = (current_ratio >= 1) & (debt_ratio <= 0.8)
common_3 = (total_debt / df['CurrentAssets']) < 1.5
# 2. Avoid low margin and value trap
common_4 = capitalization_price > 1000000000
common_5 = (capitalization_price > 6000000000) & (df['Avg_Gross_margin'] > 10)
common_6 = (capitalization_price <= 6000000000) & ((dividend_rate > 0.01) | (buyback_rate > 0.01))

In [27]:
# 1st set of conditions: Stalwart
condition_1 = (df['Avg_Gross_margin'] > 20) & (df['Avg_ebit_margin'] > 15)
condition_2 = df['Avg_sales_growth'] >= 0.05

In [28]:
# 2nd set of conditions: Asset Play
asset_1 = (core_lcr >= 1) & (lcr >= 1.2)
asset_2 = (dividend_rate > 0.05) | (buyback_rate > 0.05)
asset_3 = (current_ratio >= 1.5) & (debt_ratio <= 0.4)

#Step 3. Output

Filter the dataset using the above conditions

In [29]:
# filtered by common conditions
common_df = df
common_df['EV'] = enterprise_value
common_df['Dividend rate'] = dividend_rate
common_df['Buyback rate'] = buyback_rate
common_df['PPE_multiple'] = ppe_multiple
common_df = common_df.loc[common_1 & common_2 & common_3 & common_4 & (common_5 | common_6)]
display(common_df)

,Ticker,Name,Exchange,Price,Price_currency,Shares,Reporting_Currency,Fx_rate,Dividend,Buyback,...,InterestExpense,NetIncomeCommonStockholders,NetMargin,Avg_net_margin,Avg_NetIncome_growth,Years_of_data,EV,Dividend rate,Buyback rate,PPE_multiple
2,000088.SZ,SHN YAN TIAN PORT,SHZ,4.990,CNY,2249159936,CNY,1.000000,0.000000,0.000000,...,7.878430e+07,4.614137e+08,67.87,67.400000,13.39,3.0,8.186662e+09,0.000000,0.000000,10.342827
9,000403.SZ,PACIFIC SHUANGLIN,SHZ,22.910,CNY,732360000,CNY,1.000000,0.000000,0.000000,...,9.986722e+06,3.910640e+08,19.83,18.353333,63.10,3.0,1.502242e+10,0.000000,0.000000,0.601552
11,000408.SZ,ZANGGE MINING CO L,SHZ,26.770,CNY,1580440064,CNY,1.000000,0.000000,0.000000,...,4.802218e+04,1.427340e+09,39.40,22.230000,248.93,3.0,3.680143e+10,0.000000,0.000000,0.842274
14,000423.SZ,DONG-E-E-JIAO CO,SHZ,40.830,CNY,654022016,CNY,1.000000,0.000000,0.000000,...,5.704032e+06,4.404400e+08,11.44,-0.763333,403.84,3.0,1.585169e+10,0.000000,0.000000,0.547994
17,000513.SZ,LIVZON PHARMACEUTI,SHZ,33.450,CNY,625721024,CNY,1.000000,0.000000,0.000000,...,8.399723e+07,1.775683e+09,14.72,14.966667,17.58,3.0,3.875377e+09,0.000000,0.000000,0.373848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2791,X,United States Steel Corporation,NYQ,27.420,USD,234268992,USD,1.000000,0.000000,0.640290,...,3.130000e+08,4.174000e+09,20.59,1.253333,-186.68,3.0,3.780656e+09,0.000000,0.023351,0.401085
2798,YMM,Full Truck Alliance Co. Ltd.,NYQ,9.425,USD,1108400000,CNY,0.147437,0.000000,2.332585,...,4.000000e+04,-4.172880e+09,-89.60,-96.776667,75.94,3.0,3.840154e+10,0.000000,0.247489,0.056621
2803,ZD,"Ziff Davis, Inc.",NMS,82.800,USD,47191600,USD,1.000000,0.000000,1.659766,...,7.202300e+07,4.963880e+08,35.04,20.266667,100.26,3.0,3.668737e+09,0.000000,0.020045,0.152068
2804,ZG,"Zillow Group, Inc.",NMS,39.630,USD,58197100,USD,1.000000,0.000000,5.194365,...,1.919100e+08,-5.277770e+08,-6.48,-7.486667,89.32,3.0,-1.280649e+09,0.000000,0.131072,0.047623


In [30]:
# fitlered by 1st set of conditions: Stalwart
df_1 = common_df
df_1 = df_1.loc[condition_1 & condition_2]
print(df_1.to_string())

         Ticker                             Name Exchange       Price Price_currency       Shares Reporting_Currency   Fx_rate     Dividend     Buyback                    Last_fy   TotalAssets  CurrentAssets  CurrentLiabilities  CurrentDebtAndCapitalLeaseObligation  CurrentCapitalLeaseObligation  LongTermDebtAndCapitalLeaseObligation  LongTermCapitalLeaseObligation  TotalEquityGrossMinorityInterest  MinorityInterest  CashAndCashEquivalents  OtherShortTermInvestments  InvestmentProperties  LongTermEquityInvestment  InvestmentinFinancialAssets        NetPPE  TotalRevenue  Avg_sales_growth  CostOfRevenue  GrossMargin  Avg_Gross_margin  SellingGeneralAndAdministration          EBIT  EbitMargin  Avg_ebit_margin  Avg_ebit_growth  InterestExpense  NetIncomeCommonStockholders  NetMargin  Avg_net_margin  Avg_NetIncome_growth  Years_of_data            EV  Dividend rate  Buyback rate  PPE_multiple
2     000088.SZ                SHN YAN TIAN PORT      SHZ      4.9900            CNY   2249159936   

In [31]:
# filtered by 2nd set of conditions: Asset Play
df_2 = common_df
df_2 = df_2.loc[asset_1 & asset_2 & asset_3]
print(df_2.to_string())

       Ticker                             Name Exchange     Price Price_currency      Shares Reporting_Currency   Fx_rate  Dividend    Buyback                    Last_fy   TotalAssets  CurrentAssets  CurrentLiabilities  CurrentDebtAndCapitalLeaseObligation  CurrentCapitalLeaseObligation  LongTermDebtAndCapitalLeaseObligation  LongTermCapitalLeaseObligation  TotalEquityGrossMinorityInterest  MinorityInterest  CashAndCashEquivalents  OtherShortTermInvestments  InvestmentProperties  LongTermEquityInvestment  InvestmentinFinancialAssets        NetPPE  TotalRevenue  Avg_sales_growth  CostOfRevenue  GrossMargin  Avg_Gross_margin  SellingGeneralAndAdministration          EBIT  EbitMargin  Avg_ebit_margin  Avg_ebit_growth  InterestExpense  NetIncomeCommonStockholders  NetMargin  Avg_net_margin  Avg_NetIncome_growth  Years_of_data            EV  Dividend rate  Buyback rate  PPE_multiple
309   0194.HK                   LIU CHONG HING      HKG     7.250            HKD   378583008                H

In [32]:
# combine the results
result_set = pd.concat([df_1, df_2])
result_set = result_set.groupby("Ticker").first()
result_set = result_set.sort_values(by=['PPE_multiple', 'EV'], ascending=[True, True]).reset_index()
print(result_set.to_string())

        Ticker                             Name Exchange       Price Price_currency       Shares Reporting_Currency   Fx_rate     Dividend     Buyback                    Last_fy   TotalAssets  CurrentAssets  CurrentLiabilities  CurrentDebtAndCapitalLeaseObligation  CurrentCapitalLeaseObligation  LongTermDebtAndCapitalLeaseObligation  LongTermCapitalLeaseObligation  TotalEquityGrossMinorityInterest  MinorityInterest  CashAndCashEquivalents  OtherShortTermInvestments  InvestmentProperties  LongTermEquityInvestment  InvestmentinFinancialAssets        NetPPE  TotalRevenue  Avg_sales_growth  CostOfRevenue  GrossMargin  Avg_Gross_margin  SellingGeneralAndAdministration          EBIT  EbitMargin  Avg_ebit_margin  Avg_ebit_growth  InterestExpense  NetIncomeCommonStockholders  NetMargin  Avg_net_margin  Avg_NetIncome_growth  Years_of_data            EV  Dividend rate  Buyback rate  PPE_multiple
0      0393.HK                     GLORIOUS SUN      HKG      0.7400            HKD   1527980032     

In [33]:
# Listing Location
market = 'HK' #@param ["HK","CN", "US", "JP"]

# HK only
if market == 'HK':
  exchange_condition = result_set['Exchange'] == 'HKG'
elif market == 'CN':
# A shares only
  exchange_condition = (result_set['Exchange'] == 'SHZ') | (result_set['Exchange'] == 'SHH')
elif market == 'US':
  exchange_condition = (result_set['Exchange'] == 'NMS') | (result_set['Exchange'] == 'NYQ')
else:
  exchange_condition = (result_set['Exchange'] == 'JPX')
display_set = result_set.loc[exchange_condition].sort_values(by=['PPE_multiple', 'EV'], ascending=[True, True]).reset_index()
print(display_set.to_string())


    index   Ticker             Name Exchange    Price Price_currency       Shares Reporting_Currency   Fx_rate  Dividend    Buyback                    Last_fy   TotalAssets  CurrentAssets  CurrentLiabilities  CurrentDebtAndCapitalLeaseObligation  CurrentCapitalLeaseObligation  LongTermDebtAndCapitalLeaseObligation  LongTermCapitalLeaseObligation  TotalEquityGrossMinorityInterest  MinorityInterest  CashAndCashEquivalents  OtherShortTermInvestments  InvestmentProperties  LongTermEquityInvestment  InvestmentinFinancialAssets        NetPPE  TotalRevenue  Avg_sales_growth  CostOfRevenue  GrossMargin  Avg_Gross_margin  SellingGeneralAndAdministration          EBIT  EbitMargin  Avg_ebit_margin  Avg_ebit_growth  InterestExpense  NetIncomeCommonStockholders  NetMargin  Avg_net_margin  Avg_NetIncome_growth  Years_of_data            EV  Dividend rate  Buyback rate  PPE_multiple
0       0  0393.HK     GLORIOUS SUN      HKG    0.740            HKD   1527980032                HKD  1.000000  0.060736